The purpose of this notebook is to show how to **calibrate** XRD data using pyFAI library.

See pyFAI tutorial here: (https://www.youtube.com/watch?v=j0cphX3z2mM&ab_channel=ESRFsynchrotron)
See pyFAI documents here: (https://pyfai.readthedocs.io/en/stable/)

At 28ID-2-D-tomo beamline endstation of NSLS-II, we mainly use two different area detectors:

* PerkinElmer : https://resources.perkinelmer.com/corporate/pdfs/downloads/bro_digitalxraydetectors.pdf
* Dexela 2923 : https://www.sciencedirect.com/science/article/pii/S0168900212006730

This tutorial will demonstrate calibration with pyFAI on powder XRD data collected on LaB6 and CeO2 samples.


# general imports

In [1]:
# We need to import some Python libraries here first.

import os
import sys
import shutil
import scipy

import numpy as np
import xarray as xr

import PIL # python image library for reading tiff files
import pyFAI

In [2]:
# It would be good to have interactive plots if ipympl is installed.
# See: https://matplotlib.org/ipympl/

%matplotlib inline

# importing matplotlib for plots.
import matplotlib.pyplot as plt
plt.rcParams.update({'figure.max_open_warning': 0})
plt.rcParams['figure.constrained_layout.use'] = True

# setting up the example data path

In [3]:
# We assume this notebook and its data is on Google Drive so that we can use Google Colab.
# Otherwise, replace XXX with the exact path where this notebook (000.read_research_data_into_python.ipynb)
# and its data folder (000.read_research_data_into_python.data) is.

try:
    from google.colab import drive
    drive.mount('/content/drive')
    os.chdir('/content/drive/MyDrive/SULI-2024/repositories/pySULI/notebooks/')
except Exception as exc:
    print(exc)
    os.chdir('.')

No module named 'google'


In [4]:
# Assuming 000.read_research_data_into_python.data and 000.read_research_data_into_python.ipynb are here.

here = os.getcwd()
example_data_path = '002.learn_pyFAI.data'

# navigate into example_data_path directory
os.chdir(example_data_path)

os.listdir('.')

['from_Dexela2923_detector', 'from_PerkinElmer_detector']